# Converting Images into a Region Adjacency Graph
---

## Dependencies
1. **skimage**
  - for creating labelled components from the image (skimage.measure)
  - for finding centroid of each label (skimage.measure)
  - converting the labelled image into a RAG (skimage.future.graph)
  
2. **numpy**
  - image array manipulation operations
  
3. **pickle**
  - dumping the graphs in a file
  
4. **tqdm**
  - for progress bars

5. **cv2**
  - create video files

6. **networkx**
  - for visualizing the graphs
7. **os**
  - saving images and videos
8. **scipy**
  - for developing kdtree to find lenght of shared boundary
9. **copy**
  - create a deep copy of the graphs and remove their edge 
  - weights for more visual plotting
10. **pathlib**
  - listing all the images and trajectories
  

In [3]:
from skimage import measure, io, draw
from skimage.color import rgb2gray
from skimage.future import graph
import numpy as np
import pickle
from tqdm.notebook import tqdm
import cv2
import networkx as nx
import os
from scipy.spatial import KDTree
import copy
from pathlib import Path
from helper import *
%load_ext autoreload
%aimport helper
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Dev Dependencies

In [4]:
# %matplotlib notebook
from matplotlib import pyplot as plt

plt.set_cmap('gray')
import networkx
from IPython.core.debugger import set_trace

<Figure size 432x288 with 0 Axes>

---

## Utility Functions

#### Draw RAG edges and nodes over the image

In [5]:
def draw_edges(img, rag):
    edge_img = img.copy() # to avoid modifying the original image
    for edge in rag.edges:
        # get the node pair for the edge
        node_1, node_2 = edge
        
        # get the cordinated of the centroid of the node
        x1, y1 = map(int, rag.nodes[node_1]['centroid'])
        x2, y2 = map(int, rag.nodes[node_2]['centroid'])
        
        # draw the line joining the centroid of the 2 nodes
        line_x, line_y = draw.line(x1,y1,x2,y2)
        edge_img[line_x, line_y] = 127
 
    return edge_img

In [6]:
def draw_nodes(img, rag):
    node_img = img.copy() # to avoid modifying the original image
    for node in rag.nodes(data=True):
        
        # get the coordinates of the centroid of the node
        node_x, node_y = node[1]['centroid']
        
        # draw the circle at the centroid of the node
        circle_x, circle_y = draw.circle(node_x, node_y, 2)
        node_img[circle_x-1, circle_y-1] = 127
 
    return node_img

#### Save graph overlay images
Create output images containing the original image, labeled_image and graph overlayed over labeled image

In [7]:
def generate_overalay_graph_images(img, label_img, graph_img, img_name, trajectory_name):
    """
    Style 1 has 3 images stacked over each other. 
    The original image, the label image(where each blob has a different label) 
    and graph overlay image.
    Args:
        img: original grayscale image
        label_img: labelled_image
        graph_img: label_img with graph pverlay on it
        img_name: name of tghe image
        combined: store each trajectory images in a
                  separate folder if False. Default=True.
    Returns:
        Boolean describing the success of the operation
    """
    #combined_img = np.concatenate([img,label_img,graph_img], axis=0)
    
    fig = plt.figure(figsize=(10,10))
    
    # plot the original image
    ax = fig.add_subplot(3, 1, 1)
    ax.imshow(img, cmap="gray")
    ax.set_title("Original Image")
    
    # Plot the labeled Image
    ax = fig.add_subplot(3, 1, 2)
    ax.imshow(label_img, cmap="nipy_spectral")
    ax.set_title("Labelled Image")

    # Plot the Graph Overlay Image
    ax = fig.add_subplot(3, 1, 3)
    ax.imshow(graph_img, cmap="nipy_spectral")
    ax.set_title("Graph Overlay Image")
    
    title = f"{trajectory_name} #{int(img_name)%80} ({img_name})"
    
    fig.suptitle(title, fontsize=15, y=0.03)    
    plt.tight_layout(rect=[0, 0.05, 1, 1])
    
    target_dir = Path("/home/namit/codes/Entropy-Isomap/outputs/non_periodic/graph_overlay/")/trajectory_name
    
    # create if path does not exist
    target_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(target_dir/(img_name+".png"))
    
    plt.close()   


#### Save graph videos
Create a video of the saved images

In [8]:
def generate_overalay_graph_video(source_dir, target_dir):
    """
    Find all the images in a given 
    directory and create a video of them.
    Args:
        dir: path to the directory where all the images
             are stored
    """
    # read all the images from the source directory
    source_dir += "/*.jpg" #asuming files are always written in jpg
    imgs = io.imread_collection(source_dir, conserve_memory=True)
    
    # create a video writer in the target location
    height, width, layers = imgs[0].shape
    size = (width,height)
    target_dir += "/graph_overlay.avi"
    out = cv2.VideoWriter(target_dir,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    # write the frames to the video writer 
    for img in imgs:
        out.write(img)
        out.write(img)
    out.release()
    

#### Save networkx graph visualization images
Graphs created using networkx's draw_networkx function

In [9]:
def generate_graph_visualization_images(graphs, trajectory_name, combined=True):
    """
    Save graph visualizations as images.
    Args:
        graphs: list of graphs to be visualized
        combined: if False, directory name will
                  be determined based on index number 
                  of the graph. Default=True.
    """
    root_nodes = []
    for graph_num,gg in enumerate(tqdm(graphs)):
        
        fig,axes = plt.subplots()
        
        # setting node size
        node_size = [i[1]['area'] for i in gg.nodes(data=True)]
        sum_node_size = sum(node_size)
        node_size_normalized = [(i/sum_node_size)*5000 for i in node_size]
        
        # setting node color
        node_color = []

        for i in gg.nodes(data=True):
            current_color = i[1]['color']
            if current_color == 1:
                # this is white
                # set to light grey
                node_color.append(np.array([0.7,0.7,0.7]))
            elif current_color == 0:
                # this is black
                # set to dark grey
                node_color.append(np.array([0.3,0.3,0.3]))
            else:
                # this should never happen
                print("Unknown color of node.")
        
        # setting node label
        node_labels = {}
        for index, size in enumerate(node_size):
            node_labels[index+1] = f"{size} ({index+1})"
            
        # setting node edge colors
        edgecolors = ['k']*len(node_color)
        root_node = get_max_degree_node_color(gg)
        print(f"{graph_num} - {root_node}")
        try:
            edgecolors[root_node-1] = 'y'
        except:
            nx.draw_kamada_kawai(graph)
            return None
        root_nodes.append(root_node)
        
        #set_trace()
        
        # create the graph and save it
        aa = nx.draw_kamada_kawai(gg, 
                             node_size   = node_size_normalized, 
                             node_color  = node_color,
                             edgecolors  = edgecolors,
                             labels      = node_labels,
                             with_labels = True,
                             ax          = axes)
        
        target_dir = Path("/home/namit/codes/Entropy-Isomap/outputs/non_periodic/graph_visualization/")/trajectory_name
    
        # create if path does not exist
        target_dir.mkdir(parents=True, exist_ok=True)
        
        title = trajectory_name+f" #{graph_num%80} ({graph_num})"
        plt.title(title, y=-0.1)
        
        plt.savefig(target_dir/(title+".png"))
        
        plt.cla()
        plt.close()
    return root_nodes
    

---

## Generate Region Adjacency Graph (RAG)

#### Steps
1. Read the image, convert to grescale and thresh
2. Assign labels to every black and white region(blob).
3. Generate the Region Adjacency Graph(RAG).
4. Add other attributes to the labels such as node area, color and centroid.
5. Add weight of edges based on length of boundary shared between the 2 regions

*Adaptive thresholding is not a good idea here because we need a constant threshold for all images as we will be calculating similarities in consecutive threshed images*

In [10]:
def create_rag(imgs, trajectory_name):
    graphs = []
    kdtree = {}
    for index,img in enumerate(tqdm(imgs), start=0):

        # convert to grayscale and threshold
        gray_img = rgb2gray(img)
        thresh_img = gray_img > 0.5

        # generate disinct labels for every region
        # background set to -1 so that black blobs 
        # do not neglected and are assigned a label
        label_img = measure.label(thresh_img, background=-1)

        # generate the Region Adjacency Graph
        rag = graph.RAG(label_img)

        # RAG doesn't add a node if there is only 
        # one label in label_image
        if len(np.unique(label_img)) == 1:
            rag.add_node(1)

        # add centroid, area and color attributes 
        # to each node using regionprops
        regions = measure.regionprops(label_img, thresh_img)
        for region in regions:
            rag.nodes[region['label']] ['area'] = region['area']
            rag.nodes[region['label']] ['color'] = region['mean_intensity']
            rag.nodes[region['label']] ['centroid'] = region['centroid']
            rag.nodes[region['label']] ['perimeter'] = region['perimeter']
            # will be used to count the number pixels shared on the boundary of 2 regions
            kdtree[region['label']] = KDTree(region.coords)

        # add weight of edges
        # weight = number of neghbouring pixels 
        # of the two regions/nodes
        for edge in rag.edges:
            node1, node2 = edge
            rag[node1][node2]['weight'] = kdtree[node1].count_neighbors(kdtree[node2], 1)



        graph_img = draw_edges(draw_nodes(label_img, rag), rag)

        generate_overalay_graph_images(img, label_img, graph_img, str(index), trajectory_name)
        #generate_overalay_graph_images(img, label_img, graph_img, str(index), combined=False)

        graphs.append(rag)
    return graphs

In [9]:
# read all the images
img_filenames = ["../outputs/images/"+str(i)+".jpg" for i in range(1,1281)]
imgs = io.imread_collection(img_filenames, conserve_memory=True)
graphs = create_rag(imgs)
pickle.dump(graphs, open('../outputs/pickle/graphs.file', 'wb'))

TypeError: create_rag() missing 1 required positional argument: 'trajectory_name'

In [11]:
# read all the images
source_dir = Path("/home/namit/codes/Entropy-Isomap/outputs/images_non_periodic/")
output_dir = Path("/home/namit/codes/Entropy-Isomap/outputs/graphs/")

for trajectory in tqdm(source_dir.iterdir()):
    print(trajectory.name)
    img_filenames = sorted([str(x) for x in trajectory.iterdir()])
    imgs = io.imread_collection(img_filenames, conserve_memory=True)
    graphs = create_rag(imgs, trajectory.name)
    
    pickle.dump(graphs, open(output_dir/(trajectory.name+'.file'), 'wb'))

BR0.54-CHI2.6-R5


KeyboardInterrupt: 

Generate video of all points combined and individual trajectories

In [53]:
#generate_overalay_graph_video("../outputs/graph_overlay_combined_images/", 
#                              "../outputs/graph_overlay_combined_videos/")

# write the code for indivisual trajectories if it is needed

Generate Networkx visualizations for each RAG

(create a deepcopy of all the graphs first and remove their edges for plotting more understandable graphs)

In [27]:
for trajectory in tqdm(output_dir.iterdir()):
    print(trajectory.name)
    if trajectory.name[:13] == 'BR0.56-CHI2.8':
        graphs = pickle.load(open(trajectory, 'rb'))[:80]
        # create a deep copy
        print("> creating deep copy")
        graphs_without_edge_weights = copy.deepcopy(graphs)
        print("> Done\n")

        print("> removing edge weights")
        # remove all the edge weights of the copy
        for graph in tqdm(graphs_without_edge_weights):
            for edge in graph.edges:
                node1, node2 = edge
                del graph[node1][node2]['weight']


        print("> generating graph visualization images")
        generate_graph_visualization_images(graphs_without_edge_weights, trajectory.name)

BR0.54-CHI2.6-R2.file
BR0.50-CHI2.4-R3.file
BR0.54-CHI3.0-R5.file
BR0.52-CHI2.4-R2.file
BR0.54-CHI2.4-R4.file
BR0.56-CHI2.4-R4.file
BR0.50-CHI2.8-R2.file
BR0.56-CHI3.0-R1.file
BR0.56-CHI3.0-R4.file
BR0.56-CHI2.8-R4.file
> creating deep copy
> Done

> removing edge weights



> generating graph visualization images


0 - 2
1 - 1
2 - 2
3 - 6
4 - 4
5 - 11
6 - 10
7 - 9
8 - 9
9 - 2
10 - 2
11 - 2
12 - 2
13 - 2
14 - 2
15 - 2
16 - 2
17 - 2
18 - 2
19 - 2
20 - 2
21 - 2
22 - 2
23 - 2
24 - 2
25 - 2
26 - 2
27 - 2
28 - 2
29 - 2
30 - 2
31 - 2
32 - 2
33 - 2
34 - 2
35 - 2
36 - 2
37 - 2
38 - 2
39 - 2
40 - 2
41 - 2
42 - 2
43 - 2
44 - 2
45 - 2
46 - 2
47 - 2
48 - 2
49 - 2
50 - 2
51 - 2
52 - 2
53 - 2
54 - 2
55 - 1
56 - 1
57 - 1
58 - 1
59 - 1
60 - 1
61 - 1
62 - 1
63 - 1
64 - 1
65 - 1
66 - 1
67 - 1
68 - 1
69 - 1
70 - 1
71 - 1
72 - 1
73 - 1
74 - 1
75 - 1
76 - 1
77 - 1
78 - 1
79 - 1

BR0.52-CHI3.0-R3.file
BR0.56-CHI2.6-R5.file
BR0.56-CHI2.4-R1.file
BR0.52-CHI2.6-R4.file
BR0.50-CHI3.0-R3.file
BR0.54-CHI2.6-R4.file
BR0.54-CHI3.0-R3.file
BR0.52-CHI2.4-R4.file
BR0.50-CHI2.4-R5.file
BR0.56-CHI2.6-R3.file
BR0.50-CHI2.8-R3.file
BR0.52-CHI2.8-R3.file
BR0.56-CHI2.4-R5.file
BR0.52-CHI2.4-R5.file
BR0.56-CHI3.0-R2.file
BR0.54-CHI3.0-R4.file
BR0.54-CHI2.8-R5.file
BR0.54-CHI2.6-R1.file
BR0.56-CHI3.0-R5.file
BR0.54-CHI2.4-R1.file
BR0.50-


> generating graph visualization images


0 - 1
1 - 1
2 - 2
3 - 2
4 - 8
5 - 1
6 - 1
7 - 1
8 - 1
9 - 1
10 - 1
11 - 1
12 - 1
13 - 1
14 - 1
15 - 1
16 - 1
17 - 1
18 - 1
19 - 1
20 - 3
21 - 3
22 - 3
23 - 3
24 - 3
25 - 3
26 - 3
27 - 3
28 - 3
29 - 3
30 - 3
31 - 2
32 - 2
33 - 2
34 - 2
35 - 2
36 - 2
37 - 2
38 - 2
39 - 2
40 - 2
41 - 2
42 - 2
43 - 2
44 - 2
45 - 2
46 - 2
47 - 2
48 - 2
49 - 2
50 - 2
51 - 2
52 - 2
53 - 2
54 - 2
55 - 2
56 - 2
57 - 2
58 - 2
59 - 2
60 - 2
61 - 2
62 - 2
63 - 2
64 - 2
65 - 2
66 - 2
67 - 2
68 - 2
69 - 2
70 - 2
71 - 2
72 - 2
73 - 2
74 - 2
75 - 2
76 - 2
77 - 2
78 - 2
79 - 1

BR0.54-CHI2.4-R2.file
BR0.52-CHI2.4-R3.file
BR0.54-CHI3.0-R1.file
BR0.50-CHI2.4-R4.file
BR0.50-CHI2.6-R5.file
BR0.56-CHI2.8-R3.file
> creating deep copy
> Done

> removing edge weights


...

0 - 1
1 - 5
2 - 2
3 - 4
4 - 15
5 - 2
6 - 2
7 - 2
8 - 14
9 - 7
10 - 7
11 - 2
12 - 1
13 - 1
14 - 1
15 - 1
16 - 1
17 - 1
18 - 1
19 - 1
20 - 1
21 - 1
22 - 1
23 - 1
24 - 1
25 - 1
26 - 1
27 - 1
28 - 1
29 - 1
30 - 1
31 - 1
32 - 1
33 - 1
34 - 1
35 - 1
36 - 1
37 - 1
38 - 1
39 - 1
40 - 1
41 - 1
42 - 1
43 - 1
44 - 8
45 - 8
46 - 8
47 - 8
48 - 1
49 - 1
50 - 2
51 - 8
52 - 8
53 - 8
54 - 8
55 - 2
56 - 2
57 - 2
58 - 2
59 - 2
60 - 2
61 - 2
62 - 2
63 - 2
64 - 2
65 - 2
66 - 2
67 - 2
68 - 2
69 - 2
70 - 2
71 - 7
72 - 2
73 - 8
74 - 8
75 - 8
76 - 8
77 - 8
78 - 8
79 - 8

BR0.54-CHI3.0-R2.file
BR0.56-CHI2.8-R5.file
> creating deep copy
> Done

> removing edge weights



> generating graph visualization images


0 - 2
1 - 4
2 - 7
3 - 1
4 - 1
5 - 1
6 - 1
7 - 1
8 - 1
9 - 1
10 - 1
11 - 1
12 - 1
13 - 1
14 - 1
15 - 1
16 - 1
17 - 1
18 - 1
19 - 1
20 - 1
21 - 1
22 - 1
23 - 1
24 - 1
25 - 1
26 - 1
27 - 1
28 - 1
29 - 1
30 - 1
31 - 1
32 - 1
33 - 1
34 - 1
35 - 1
36 - 1
37 - 1
38 - 1
39 - 1
40 - 1
41 - 1
42 - 1
43 - 1
44 - 1
45 - 1
46 - 1
47 - 1
48 - 1
49 - 1
50 - 1
51 - 1
52 - 1
53 - 1
54 - 1
55 - 1
56 - 1
57 - 1
58 - 1
59 - 1
60 - 1
61 - 1
62 - 1
63 - 1
64 - 1
65 - 1
66 - 1
67 - 1
68 - 1
69 - 1
70 - 1
71 - 1
72 - 1
73 - 1
74 - 1
75 - 1
76 - 1
77 - 1
78 - 1
79 - 1

BR0.52-CHI2.8-R2.file
BR0.54-CHI2.8-R3.file
BR0.52-CHI2.8-R5.file
BR0.52-CHI2.8-R1.file
BR0.50-CHI3.0-R5.file
BR0.50-CHI2.8-R4.file
BR0.52-CHI3.0-R2.file
BR0.56-CHI3.0-R3.file
BR0.54-CHI2.4-R3.file
BR0.56-CHI2.6-R4.file
BR0.50-CHI2.6-R4.file
BR0.52-CHI2.6-R2.file
BR0.56-CHI2.6-R2.file
BR0.54-CHI2.8-R1.file
BR0.52-CHI2.6-R5.file
BR0.54-CHI2.6-R3.file
BR0.50-CHI2.6-R2.file
BR0.50-CHI2.6-R3.file
BR0.52-CHI3.0-R5.file
BR0.50-CHI2.6-R1.file
BR0.54-CH


> generating graph visualization images


0 - 3
1 - 11
2 - 2
3 - 3
4 - 11
5 - 10
6 - 9
7 - 3
8 - 1
9 - 1
10 - 1
11 - 1
12 - 1
13 - 1
14 - 1
15 - 1
16 - 1
17 - 1
18 - 1
19 - 1
20 - 1
21 - 1
22 - 1
23 - 1
24 - 1
25 - 1
26 - 1
27 - 1
28 - 1
29 - 1
30 - 1
31 - 1
32 - 1
33 - 1
34 - 1
35 - 1
36 - 1
37 - 1
38 - 1
39 - 1
40 - 1
41 - 1
42 - 1
43 - 1
44 - 1
45 - 3
46 - 1
47 - 1
48 - 1
49 - 1
50 - 1
51 - 1
52 - 1
53 - 1
54 - 1
55 - 1
56 - 1
57 - 1
58 - 2
59 - 2
60 - 2
61 - 2
62 - 2
63 - 2
64 - 2
65 - 2
66 - 2
67 - 2
68 - 2
69 - 2
70 - 2
71 - 2
72 - 2
73 - 2
74 - 2
75 - 2
76 - 2
77 - 2
78 - 2
79 - 2

BR0.52-CHI2.4-R1.file
BR0.56-CHI2.4-R3.file
BR0.52-CHI3.0-R1.file
BR0.56-CHI2.4-R2.file
BR0.50-CHI2.8-R5.file
BR0.52-CHI2.6-R1.file
BR0.54-CHI2.8-R4.file
BR0.52-CHI2.8-R4.file
BR0.50-CHI2.8-R1.file
BR0.56-CHI2.6-R1.file
BR0.50-CHI3.0-R1.file
BR0.50-CHI3.0-R2.file
BR0.50-CHI2.4-R1.file
BR0.54-CHI2.4-R5.file
BR0.52-CHI3.0-R4.file
BR0.50-CHI2.4-R2.file



---

In [4]:
graphs = pickle.load(open("../outputs/pickle/graphs.file", 'rb'))